In [22]:
import folium
from geopy.geocoders import Nominatim
import webbrowser
import requests
from requests.structures import CaseInsensitiveDict
import pandas as pd
from sklearn.cluster import KMeans
from geopy.distance import geodesic

GEOAPIFY_API_KEY = "YOUR_GEOAPIFY_API_KEY"

city = str(input("Enter the name of the city that you plan to visit: "))
limit_results = 20

geolocator = Nominatim(user_agent="MyApp")
location = geolocator.geocode(city)
my_map = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

city_to_place_id_url = f"https://api.geoapify.com/v1/geocode/search?text={city}&apiKey={GEOAPIFY_API_KEY}"
headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
resp = requests.get(city_to_place_id_url, headers=headers)
result_city = resp.json()
country = result_city['features'][0]['properties']['address_line2']

data = []
adding_places = True

while adding_places:
    place = str(input('Enter the name of a place you want to visit. Type "None" if there are no more places: '))
    if place == "None":
        adding_places = False
    else:
        place.replace(' ', '%20')
        location_to_coordinates_url = f"https://api.geoapify.com/v1/geocode/search?text={place}%2C%20{city}%2C%20{country}&format=json&apiKey={GEOAPIFY_API_KEY}"
        headers = CaseInsensitiveDict()
        headers["Accept"] = "application/json"
        resp = requests.get(location_to_coordinates_url, headers=headers)
        result = resp.json()['results'][0]
        name = result.get('name') or place
        address = result.get('address_line2')
        coordinates = [result['lon'], result['lat']]
        data.append({
            'Name': name,
            'Address': address,
            'Coordinates': coordinates
        })

df = pd.DataFrame(data)

for row in df.itertuples(index=False):
    name = row.Name
    address = row.Address
    coordinates = row.Coordinates
    marker = folium.Marker(
        location=[coordinates[1], coordinates[0]],
        icon=folium.Icon(color='blue')
    )
    popup_html = f"""
    <div style="width: 300px;">
        <h3>{name}</h3>
        <p><strong>Address:</strong> {address}</p>
    </div>
    """
    folium.Popup(popup_html, max_width=400).add_to(marker)
    marker.add_to(my_map)

coordinates = df['Coordinates'].tolist()
kmeans = KMeans(n_clusters=1, random_state=0).fit(coordinates)
center = kmeans.cluster_centers_[0]

df['Distance'] = df['Coordinates'].apply(lambda x: geodesic(center, x).kilometers)
max_distance = df['Distance'].max()

circle_radius = 250  # In meters
circle_color = '#92ea80'  # Lighter shade of green

folium.Circle(
    location=[center[1], center[0]],
    radius=circle_radius,
    color=circle_color,
    fill=True,
    fill_color=circle_color
).add_to(my_map)

marker = folium.Marker(
    location=[center[1], center[0]],
    icon=folium.Icon(color='green')
)

popup_html = f"""
<div style="width: 300px;">
    <h3>Best Location for Accommodation</h3>
    <p><strong>Find your ideal accommodation around this area!</strong></p>
    <p><strong>Location:</strong> {center[1]}, {center[0]}</p>
</div>
"""

folium.Popup(popup_html, max_width=400).add_to(marker)
marker.add_to(my_map)

my_map.save("maps/map.html")
webbrowser.open("maps/map.html")

print("Best accommodation area:", center)

Enter the name of the city that you plan to visit:  Helsinki
Enter the name of a place you want to visit. Type "None" if there are no more places:  Train station
Enter the name of a place you want to visit. Type "None" if there are no more places:  Helsinki cathedral
Enter the name of a place you want to visit. Type "None" if there are no more places:  vanha kauppahalli
Enter the name of a place you want to visit. Type "None" if there are no more places:  None


/home/cargonvis/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Best accommodation area: [24.94859452 60.16915982]
